In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
X = data.drop('DEATH_EVENT', axis=1).values
y = data.DEATH_EVENT.values

---

In [ ]:
pd.DataFrame(data.corr().DEATH_EVENT[:-1]).sort_values(by='DEATH_EVENT', ascending=False)

In [ ]:
new = data.ejection_fraction * data.time
pd.DataFrame(np.concatenate([(new).values.reshape(-1, 1), data.DEATH_EVENT.values.reshape(-1, 1)], axis=1)).corr()

In [ ]:
new_columns = [
    data.serum_creatinine/data.time, # 0.375363
    data.serum_creatinine/data.ejection_fraction, # 0.39516
    data.serum_creatinine/data.serum_sodium, # 0.29928
    data.age/data.serum_sodium, # 0.280502
    data.age/data.ejection_fraction, # 0.432706
    data.age/data.time, # 0.470318
    data.serum_creatinine * data.age, # 0.327823
    data.serum_sodium * data.ejection_fraction, # -0.279538
    data.serum_sodium * data.time, # -0.531718
    data.ejection_fraction * data.time, # -0.559566    
]
new_arrays = np.array(data.DEATH_EVENT.values.reshape(-1, 1))
for new_column in new_columns:
    new_arrays = np.concatenate([new_arrays, new_column.values.reshape(-1, 1)], axis=1)
X = np.concatenate([X, new_arrays], axis=1)

---

In [ ]:
# no need to modify extremely high or low values

# fig, axs = plt.subplots(3, 4, figsize=(20, 14))
# columns = data.columns
# k = 0
# for i in range(3):
#     for j in range(4):
#         axs[i, j].plot(data[columns[k]], label=columns[k])
#         axs[i, j].legend()
#         k += 1
# plt.show()

In [ ]:
# no need to encode

# data.info()

---

In [ ]:
from sklearn.decomposition import KernelPCA
pca = KernelPCA(n_components=round(X.shape[1]/5), kernel='rbf')
X = np.concatenate([X, pca.fit_transform(X)], axis=1)

---

In [ ]:
from sklearn.preprocessing import Normalizer
sc = Normalizer()
X = sc.fit_transform(X)

---

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xvalid, ytrain, yvalid = train_test_split(X, y, test_size=.2, random_state=333)

---

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=1000)
model.fit(Xtrain, ytrain)
ypred = model.predict(Xvalid)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(yvalid, ypred), confusion_matrix(yvalid, ypred)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(estimator=XGBClassifier(n_estimators=1000), X=X, y=y, scoring='accuracy', cv=5, n_jobs=-1)